# AI-Powered Personalized Exam Preparation Assistant for Competitive Exams
## Machine Learning Laboratory Experiments (1-9)

**Project**: TNPSC Exam Preparation Assistant  
**Course**: 21CSC305P Machine Learning Lab  
**Experiments**: 9 ML Programs for Competitive Exam Analysis

---

## Setup and Imports

In [ ]:
# Import all required libraries
import os
import re
import json
import time
import random
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Import our TNPSC Assistant
from tnpsc_assistant import TNPSCExamAssistant, UNITS, load_questions, load_syllabus

print("All libraries imported successfully!")
print("="*60)
print("AI-POWERED TNPSC EXAM PREPARATION ASSISTANT")
print("MACHINE LEARNING LABORATORY EXPERIMENTS")
print("="*60)

## Initialize TNPSC Assistant and Load Data

In [ ]:
# Initialize the TNPSC Assistant
assistant = TNPSCExamAssistant()
questions_df = assistant.questions_df
syllabus = assistant.syllabus

print(f"Dataset loaded with {len(questions_df)} questions")
print(f"Units available: {list(UNITS.keys())}")
print(f"Columns: {list(questions_df.columns)}")

---
# EXPERIMENT 1: Load and View TNPSC Dataset
**AIM**: To implement a program to load and view the TNPSC questions dataset

In [ ]:
print("EXPERIMENT 1: LOAD AND VIEW TNPSC DATASET")
print("="*50)

print("ALGORITHM:")
print("1. Load TNPSC questions dataset")
print("2. Create a duplicate dataset for safety")
print("3. Display basic information about the dataset")
print("4. Show sample questions and statistics")

print("\nPROGRAM EXECUTION:")

# Load the dataset
print(f"Original dataset shape: {questions_df.shape}")

# Create duplicate
tnpsc_data = questions_df.copy()

# Display basic info
print(f"\nDataset columns: {list(tnpsc_data.columns)}")
print(f"Total questions: {len(tnpsc_data)}")
print(f"Units covered: {sorted(tnpsc_data['unit'].unique())}")

# Display first 10 questions
print("\nFirst 10 questions:")
display_cols = ['unit', 'question', 'difficulty', 'year']
display(tnpsc_data[display_cols].head(10))

# Unit distribution
print("\nQuestions per unit:")
unit_counts = tnpsc_data['unit'].value_counts().sort_index()
for unit, count in unit_counts.items():
    print(f"Unit {unit} ({UNITS[unit]}): {count} questions")

print("\nRESULT: TNPSC dataset loaded and analyzed successfully!")

---
# EXPERIMENT 2: Dataset Statistics
**AIM**: To display the summary and statistics of the TNPSC questions dataset

In [ ]:
print("EXPERIMENT 2: DATASET STATISTICS")
print("="*50)

print("ALGORITHM:")
print("1. Load TNPSC dataset")
print("2. Calculate descriptive statistics")
print("3. Display difficulty distribution")
print("4. Show year-wise question distribution")

print("\nPROGRAM EXECUTION:")

# Basic statistics
print("Dataset Overview:")
print(f"Total Questions: {len(questions_df)}")
print(f"Units: {len(questions_df['unit'].unique())}")
print(f"Years covered: {sorted(questions_df['year'].unique())}")

# Difficulty distribution
print("\nDifficulty Distribution:")
difficulty_counts = questions_df['difficulty'].value_counts()
for diff, count in difficulty_counts.items():
    percentage = (count / len(questions_df)) * 100
    print(f"{diff.capitalize()}: {count} ({percentage:.1f}%)")

# Visualizations
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Unit distribution
unit_counts = questions_df['unit'].value_counts().sort_index()
axes[0,0].bar(unit_counts.index, unit_counts.values)
axes[0,0].set_title('Questions per Unit')
axes[0,0].set_xlabel('Unit')
axes[0,0].set_ylabel('Number of Questions')

# Difficulty distribution
difficulty_counts.plot(kind='pie', ax=axes[0,1], autopct='%1.1f%%')
axes[0,1].set_title('Difficulty Distribution')

# Year distribution
year_counts = questions_df['year'].value_counts().sort_index()
axes[1,0].bar(year_counts.index, year_counts.values)
axes[1,0].set_title('Questions by Year')
axes[1,0].set_xlabel('Year')
axes[1,0].set_ylabel('Number of Questions')

# Unit vs Difficulty heatmap
unit_difficulty = pd.crosstab(questions_df['unit'], questions_df['difficulty'])
sns.heatmap(unit_difficulty, annot=True, fmt='d', ax=axes[1,1])
axes[1,1].set_title('Unit vs Difficulty Distribution')

plt.tight_layout()
plt.show()

print("\nRESULT: Dataset statistics calculated and displayed successfully!")

---
# EXPERIMENT 3: Linear Regression Prediction
**AIM**: To implement linear regression to predict question difficulty scores

In [ ]:
print("EXPERIMENT 3: LINEAR REGRESSION PREDICTION")
print("="*50)

print("ALGORITHM:")
print("1. Prepare features from question text and metadata")
print("2. Convert difficulty to numerical scores")
print("3. Split data into training and testing sets")
print("4. Train linear regression model")
print("5. Make predictions and evaluate performance")

print("\nPROGRAM EXECUTION:")

# Prepare features
df = questions_df.copy()

# Convert difficulty to numerical scores
difficulty_map = {'easy': 1, 'medium': 2, 'hard': 3}
df['difficulty_score'] = df['difficulty'].map(difficulty_map)

# Create features
df['question_length'] = df['question'].str.len()
df['option_a_length'] = df['option_a'].str.len()
df['option_b_length'] = df['option_b'].str.len()
df['option_c_length'] = df['option_c'].str.len()
df['option_d_length'] = df['option_d'].str.len()
df['avg_option_length'] = (df['option_a_length'] + df['option_b_length'] + 
                          df['option_c_length'] + df['option_d_length']) / 4

# Features and target
features = ['unit', 'year', 'question_length', 'avg_option_length']
X = df[features]
y = df['difficulty_score']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Model Performance:")
print(f"Mean Squared Error: {mse:.4f}")
print(f"R² Score: {r2:.4f}")

# Feature importance
print(f"\nFeature Coefficients:")
for feature, coef in zip(features, model.coef_):
    print(f"{feature}: {coef:.4f}")
print(f"Intercept: {model.intercept_:.4f}")

# Visualization
plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.scatter(y_test, y_pred, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Actual Difficulty Score')
plt.ylabel('Predicted Difficulty Score')
plt.title('Actual vs Predicted')

plt.subplot(1, 3, 2)
residuals = y_test - y_pred
plt.scatter(y_pred, residuals, alpha=0.6)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted Difficulty Score')
plt.ylabel('Residuals')
plt.title('Residual Plot')

plt.subplot(1, 3, 3)
plt.bar(features, np.abs(model.coef_))
plt.title('Feature Importance (Absolute Coefficients)')
plt.xticks(rotation=45)
plt.ylabel('Absolute Coefficient Value')

plt.tight_layout()
plt.show()

print("\nRESULT: Linear regression model trained and evaluated successfully!")

---
# EXPERIMENT 4.1: Bayesian Logistic Regression
**AIM**: To implement Bayesian logistic regression for classifying questions by unit

In [ ]:
print("EXPERIMENT 4.1: BAYESIAN LOGISTIC REGRESSION")
print("="*50)

print("ALGORITHM:")
print("1. Prepare text features using TF-IDF")
print("2. Create binary classification problem (Science vs Non-Science)")
print("3. Implement Bayesian inference using sklearn approximation")
print("4. Train and evaluate the model")

print("\nPROGRAM EXECUTION:")

# Prepare data
df = questions_df.copy()

# Create binary classification: Science (Unit 1) vs Non-Science
df['is_science'] = (df['unit'] == 1).astype(int)

# Create text features using TF-IDF
vectorizer = TfidfVectorizer(max_features=100, stop_words='english')
X_text = vectorizer.fit_transform(df['question']).toarray()

# Add numerical features
df['question_length'] = df['question'].str.len()
X_numerical = df[['question_length', 'year']].values

# Combine features
X = np.hstack([X_text, X_numerical])
y = df['is_science']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train Bayesian-inspired Logistic Regression (using regularization as prior)
model = LogisticRegression(C=1.0, random_state=42, max_iter=1000)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)

# Evaluate
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")
print(f"\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['Non-Science', 'Science']))

# Visualization
plt.figure(figsize=(12, 4))

# Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
plt.subplot(1, 3, 1)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')

# Prediction Probabilities
plt.subplot(1, 3, 2)
plt.hist(y_pred_proba[:, 1], bins=20, alpha=0.7, edgecolor='black')
plt.title('Prediction Probability Distribution')
plt.xlabel('Probability of Science')
plt.ylabel('Frequency')

# ROC Curve
from sklearn.metrics import roc_curve, auc
fpr, tpr, _ = roc_curve(y_test, y_pred_proba[:, 1])
roc_auc = auc(fpr, tpr)
plt.subplot(1, 3, 3)
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")

plt.tight_layout()
plt.show()

print("\nRESULT: Bayesian logistic regression implemented and evaluated successfully!")

---
# EXPERIMENT 4.2: SVM Classification
**AIM**: To implement SVM for classifying question difficulty levels

In [ ]:
print("EXPERIMENT 4.2: SVM CLASSIFICATION")
print("="*50)

print("ALGORITHM:")
print("1. Prepare features from question text and metadata")
print("2. Create multi-class classification for difficulty levels")
print("3. Train SVM model with RBF kernel")
print("4. Evaluate model performance")

print("\nPROGRAM EXECUTION:")

# Prepare data
df = questions_df.copy()

# Create features
vectorizer = TfidfVectorizer(max_features=50, stop_words='english')
X_text = vectorizer.fit_transform(df['question']).toarray()

# Additional features
df['question_length'] = df['question'].str.len()
df['has_numbers'] = df['question'].str.contains(r'\d').astype(int)
X_numerical = df[['unit', 'year', 'question_length', 'has_numbers']].values

# Combine features
X = np.hstack([X_text, X_numerical])
y = df['difficulty']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train SVM
svm_model = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)
svm_model.fit(X_train, y_train)

# Predictions
y_pred = svm_model.predict(X_test)

# Evaluate
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")
print(f"\nClassification Report:")
print(classification_report(y_test, y_pred))

# Visualization
plt.figure(figsize=(10, 4))

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
plt.subplot(1, 2, 1)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['easy', 'hard', 'medium'], 
            yticklabels=['easy', 'hard', 'medium'])
plt.title('SVM Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')

# Prediction Distribution
plt.subplot(1, 2, 2)
pred_counts = pd.Series(y_pred).value_counts()
actual_counts = pd.Series(y_test).value_counts()
x = np.arange(len(pred_counts))
width = 0.35
plt.bar(x - width/2, actual_counts.values, width, label='Actual', alpha=0.7)
plt.bar(x + width/2, pred_counts.values, width, label='Predicted', alpha=0.7)
plt.xlabel('Difficulty Level')
plt.ylabel('Count')
plt.title('Actual vs Predicted Distribution')
plt.xticks(x, pred_counts.index)
plt.legend()

plt.tight_layout()
plt.show()

print("\nRESULT: SVM classification model trained and evaluated successfully!")

---
# EXPERIMENT 5.1: K-Means Clustering
**AIM**: To implement K-means clustering to categorize TNPSC questions

In [ ]:
print("EXPERIMENT 5.1: K-MEANS CLUSTERING")
print("="*50)

print("ALGORITHM:")
print("1. Create feature vectors from question text")
print("2. Apply K-means clustering with k=8 (number of units)")
print("3. Analyze clusters and their characteristics")
print("4. Visualize clustering results")

print("\nPROGRAM EXECUTION:")

# Prepare features
df = questions_df.copy()

# Create TF-IDF features
vectorizer = TfidfVectorizer(max_features=100, stop_words='english')
X_text = vectorizer.fit_transform(df['question']).toarray()

# Add numerical features
df['question_length'] = df['question'].str.len()
X_numerical = df[['question_length', 'year']].values

# Normalize numerical features
scaler = StandardScaler()
X_numerical_scaled = scaler.fit_transform(X_numerical)

# Combine features
X = np.hstack([X_text, X_numerical_scaled])

# Apply K-means
kmeans = KMeans(n_clusters=8, random_state=42, n_init=10)
clusters = kmeans.fit_predict(X)

# Add cluster labels to dataframe
df['cluster'] = clusters

# Analyze clusters
print("Cluster Analysis:")
for cluster_id in range(8):
    cluster_data = df[df['cluster'] == cluster_id]
    if len(cluster_data) > 0:
        most_common_unit = cluster_data['unit'].mode().iloc[0]
        most_common_difficulty = cluster_data['difficulty'].mode().iloc[0]
        
        print(f"Cluster {cluster_id}: {len(cluster_data)} questions")
        print(f"  Most common unit: {most_common_unit} ({UNITS.get(most_common_unit, 'Unknown')})")
        print(f"  Most common difficulty: {most_common_difficulty}")
        print(f"  Units distribution: {dict(cluster_data['unit'].value_counts())}")
        print()

# Visualize using PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

plt.figure(figsize=(15, 5))

# Plot clusters
plt.subplot(1, 3, 1)
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=clusters, cmap='viridis', alpha=0.6)
plt.title('K-Means Clustering Results')
plt.xlabel('First Principal Component')
plt.ylabel('Second Principal Component')
plt.colorbar(scatter)

# Plot actual units
plt.subplot(1, 3, 2)
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=df['unit'], cmap='tab10', alpha=0.6)
plt.title('Actual Unit Labels')
plt.xlabel('First Principal Component')
plt.ylabel('Second Principal Component')
plt.colorbar(scatter)

# Cluster centers
plt.subplot(1, 3, 3)
centers_pca = pca.transform(kmeans.cluster_centers_)
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=clusters, cmap='viridis', alpha=0.6)
plt.scatter(centers_pca[:, 0], centers_pca[:, 1], c='red', marker='x', s=200, linewidths=3)
plt.title('K-Means with Cluster Centers')
plt.xlabel('First Principal Component')
plt.ylabel('Second Principal Component')

plt.tight_layout()
plt.show()

print("\nRESULT: K-means clustering applied successfully to TNPSC questions!")

---
# EXPERIMENT 5.2: Gaussian Mixture Models
**AIM**: To implement GMM to categorize TNPSC questions with probabilistic clustering

In [ ]:
print("EXPERIMENT 5.2: GAUSSIAN MIXTURE MODELS")
print("="*50)

# Same feature preparation as K-means
df = questions_df.copy()
vectorizer = TfidfVectorizer(max_features=100, stop_words='english')
X_text = vectorizer.fit_transform(df['question']).toarray()
df['question_length'] = df['question'].str.len()
X_numerical = df[['question_length', 'year']].values
scaler = StandardScaler()
X_numerical_scaled = scaler.fit_transform(X_numerical)
X = np.hstack([X_text, X_numerical_scaled])

# Apply Gaussian Mixture Model
gmm = GaussianMixture(n_components=8, random_state=42, covariance_type='full')
gmm.fit(X)

# Get cluster assignments and probabilities
clusters = gmm.predict(X)
probabilities = gmm.predict_proba(X)

# Add to dataframe
df['gmm_cluster'] = clusters
df['max_probability'] = np.max(probabilities, axis=1)

# Calculate AIC and BIC
aic = gmm.aic(X)
bic = gmm.bic(X)
print(f"Model Selection Metrics:")
print(f"AIC: {aic:.2f}")
print(f"BIC: {bic:.2f}")

# Visualize using PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

plt.figure(figsize=(12, 5))

# Plot GMM clusters
plt.subplot(1, 2, 1)
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=clusters, cmap='viridis', alpha=0.6)
plt.title('GMM Clustering Results')
plt.xlabel('First Principal Component')
plt.ylabel('Second Principal Component')
plt.colorbar(scatter)

# Plot probability confidence
plt.subplot(1, 2, 2)
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=df['max_probability'], 
                    cmap='plasma', alpha=0.6)
plt.title('Assignment Probability Confidence')
plt.xlabel('First Principal Component')
plt.ylabel('Second Principal Component')
plt.colorbar(scatter, label='Max Probability')

plt.tight_layout()
plt.show()

print("\nRESULT: Gaussian Mixture Model clustering applied successfully!")

---
# EXPERIMENT 6: Principal Component Analysis
**AIM**: To perform PCA on TNPSC question features for dimensionality reduction

In [ ]:
print("EXPERIMENT 6: PRINCIPAL COMPONENT ANALYSIS")
print("="*50)

# Prepare comprehensive feature set
df = questions_df.copy()
vectorizer = TfidfVectorizer(max_features=200, stop_words='english')
X_text = vectorizer.fit_transform(df['question']).toarray()

# Additional features
df['question_length'] = df['question'].str.len()
df['word_count'] = df['question'].str.split().str.len()
df['has_numbers'] = df['question'].str.contains(r'\d').astype(int)
df['has_punctuation'] = df['question'].str.contains(r'[^\w\s]').astype(int)
df['avg_word_length'] = df['question'].apply(lambda x: np.mean([len(word) for word in x.split()]))

X_numerical = df[['unit', 'year', 'question_length', 'word_count', 
                 'has_numbers', 'has_punctuation', 'avg_word_length']].values

# Combine all features
X = np.hstack([X_text, X_numerical])
print(f"Original feature dimensions: {X.shape}")

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply PCA
pca = PCA()
X_pca = pca.fit_transform(X_scaled)

# Analyze explained variance
explained_variance_ratio = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance_ratio)

print(f"Explained variance by first 10 components:")
for i in range(min(10, len(explained_variance_ratio))):
    print(f"PC{i+1}: {explained_variance_ratio[i]:.4f} ({cumulative_variance[i]:.4f} cumulative)")

# Find number of components for 95% variance
n_components_95 = np.argmax(cumulative_variance >= 0.95) + 1
print(f"\nComponents needed for 95% variance: {n_components_95}")

# Visualizations
plt.figure(figsize=(15, 10))

# Scree plot
plt.subplot(2, 3, 1)
plt.plot(range(1, min(21, len(explained_variance_ratio) + 1)), 
        explained_variance_ratio[:20], 'bo-')
plt.title('Scree Plot')
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance Ratio')
plt.grid(True)

# Cumulative variance plot
plt.subplot(2, 3, 2)
plt.plot(range(1, min(21, len(cumulative_variance) + 1)), 
        cumulative_variance[:20], 'ro-')
plt.axhline(y=0.95, color='k', linestyle='--', label='95% Variance')
plt.title('Cumulative Explained Variance')
plt.xlabel('Principal Component')
plt.ylabel('Cumulative Variance Ratio')
plt.legend()
plt.grid(True)

# 2D visualization by unit
plt.subplot(2, 3, 3)
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=df['unit'], cmap='tab10', alpha=0.6)
plt.title('PCA: Questions by Unit')
plt.xlabel('First Principal Component')
plt.ylabel('Second Principal Component')
plt.colorbar(scatter)

# 2D visualization by difficulty
plt.subplot(2, 3, 4)
difficulty_map = {'easy': 0, 'medium': 1, 'hard': 2}
difficulty_numeric = df['difficulty'].map(difficulty_map)
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=difficulty_numeric, cmap='viridis', alpha=0.6)
plt.title('PCA: Questions by Difficulty')
plt.xlabel('First Principal Component')
plt.ylabel('Second Principal Component')
plt.colorbar(scatter, ticks=[0, 1, 2], label='Difficulty')

# 2D visualization by year
plt.subplot(2, 3, 5)
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=df['year'], cmap='plasma', alpha=0.6)
plt.title('PCA: Questions by Year')
plt.xlabel('First Principal Component')
plt.ylabel('Second Principal Component')
plt.colorbar(scatter)

# Explained variance bar plot
plt.subplot(2, 3, 6)
plt.bar(range(1, min(11, len(explained_variance_ratio) + 1)), 
        explained_variance_ratio[:10])
plt.title('Explained Variance by Component')
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance Ratio')

plt.tight_layout()
plt.show()

print("\nRESULT: PCA analysis completed successfully!")

---
# EXPERIMENT 8: CART Decision Tree
**AIM**: To implement CART learning algorithm for TNPSC question categorization

In [ ]:
print("EXPERIMENT 8: CART DECISION TREE")
print("="*50)

# Prepare features
df = questions_df.copy()

# Create comprehensive features
df['question_length'] = df['question'].str.len()
df['word_count'] = df['question'].str.split().str.len()
df['has_numbers'] = df['question'].str.contains(r'\d').astype(int)
df['has_punctuation'] = df['question'].str.contains(r'[^\w\s]').astype(int)
df['avg_word_length'] = df['question'].apply(lambda x: np.mean([len(word) for word in x.split()]))
df['question_mark_count'] = df['question'].str.count(r'\?')
df['capital_letters_count'] = df['question'].str.count(r'[A-Z]')

# Text-based features using keyword matching
science_keywords = ['physics', 'chemistry', 'biology', 'science', 'atom', 'cell', 'energy']
history_keywords = ['history', 'ancient', 'medieval', 'empire', 'dynasty', 'culture']
geography_keywords = ['geography', 'river', 'mountain', 'climate', 'ocean', 'continent']
polity_keywords = ['constitution', 'government', 'parliament', 'president', 'democracy']
economy_keywords = ['economy', 'economic', 'gdp', 'inflation', 'bank', 'finance']

df['science_keywords'] = df['question'].str.lower().apply(
    lambda x: sum(1 for keyword in science_keywords if keyword in x))
df['history_keywords'] = df['question'].str.lower().apply(
    lambda x: sum(1 for keyword in history_keywords if keyword in x))
df['geography_keywords'] = df['question'].str.lower().apply(
    lambda x: sum(1 for keyword in geography_keywords if keyword in x))
df['polity_keywords'] = df['question'].str.lower().apply(
    lambda x: sum(1 for keyword in polity_keywords if keyword in x))
df['economy_keywords'] = df['question'].str.lower().apply(
    lambda x: sum(1 for keyword in economy_keywords if keyword in x))

# Features and target
feature_columns = ['year', 'question_length', 'word_count', 'has_numbers', 
                  'has_punctuation', 'avg_word_length', 'question_mark_count',
                  'capital_letters_count', 'science_keywords', 'history_keywords',
                  'geography_keywords', 'polity_keywords', 'economy_keywords']

X = df[feature_columns]
y = df['unit']  # Predict unit

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Create and train decision tree
dt_classifier = DecisionTreeClassifier(
    criterion='gini',
    max_depth=5,
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42
)

dt_classifier.fit(X_train, y_train)

# Make predictions
y_pred = dt_classifier.predict(X_test)

# Evaluate
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")
print(f"\nClassification Report:")
print(classification_report(y_test, y_pred))

# Feature importance
feature_importance = dt_classifier.feature_importances_
importance_df = pd.DataFrame({
    'feature': feature_columns,
    'importance': feature_importance
}).sort_values('importance', ascending=False)

print(f"\nFeature Importance:")
print(importance_df)

# Visualizations
plt.figure(figsize=(15, 5))

# Feature importance
plt.subplot(1, 3, 1)
top_features = importance_df.head(8)
plt.barh(range(len(top_features)), top_features['importance'])
plt.yticks(range(len(top_features)), top_features['feature'])
plt.title('Feature Importance')
plt.xlabel('Importance')

# Confusion matrix
plt.subplot(1, 3, 2)
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted Unit')
plt.ylabel('Actual Unit')

# Prediction accuracy by unit
plt.subplot(1, 3, 3)
unit_accuracy = []
for unit in sorted(y_test.unique()):
    unit_mask = y_test == unit
    unit_acc = accuracy_score(y_test[unit_mask], y_pred[unit_mask])
    unit_accuracy.append(unit_acc)

plt.bar(sorted(y_test.unique()), unit_accuracy)
plt.title('Accuracy by Unit')
plt.xlabel('Unit')
plt.ylabel('Accuracy')
plt.ylim(0, 1)

plt.tight_layout()
plt.show()

print("\nRESULT: CART decision tree classifier implemented and evaluated successfully!")

---
# EXPERIMENT 9: Ensemble Learning
**AIM**: To implement ensemble learning models for improved classification performance

In [ ]:
print("EXPERIMENT 9: ENSEMBLE LEARNING")
print("="*50)

# Use same features as decision tree
X = df[feature_columns]
y = df['unit']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 1. Random Forest (Bagging)
print("Training Random Forest...")
rf_classifier = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42
)
rf_classifier.fit(X_train, y_train)
rf_pred = rf_classifier.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_pred)

# 2. AdaBoost (Boosting)
print("Training AdaBoost...")
ada_classifier = AdaBoostClassifier(
    n_estimators=50,
    learning_rate=1.0,
    random_state=42
)
ada_classifier.fit(X_train, y_train)
ada_pred = ada_classifier.predict(X_test)
ada_accuracy = accuracy_score(y_test, ada_pred)

# 3. Single Decision Tree for comparison
print("Training Single Decision Tree...")
dt_classifier = DecisionTreeClassifier(max_depth=10, random_state=42)
dt_classifier.fit(X_train, y_train)
dt_pred = dt_classifier.predict(X_test)
dt_accuracy = accuracy_score(y_test, dt_pred)

# Compare results
print(f"\nModel Performance Comparison:")
print(f"Single Decision Tree Accuracy: {dt_accuracy:.4f}")
print(f"Random Forest Accuracy: {rf_accuracy:.4f}")
print(f"AdaBoost Accuracy: {ada_accuracy:.4f}")

# Feature importance comparison
rf_importance = rf_classifier.feature_importances_
ada_importance = ada_classifier.feature_importances_

importance_comparison = pd.DataFrame({
    'feature': feature_columns,
    'random_forest': rf_importance,
    'adaboost': ada_importance
}).sort_values('random_forest', ascending=False)

print(f"\nFeature Importance Comparison:")
print(importance_comparison.head(10))

# Visualizations
plt.figure(figsize=(15, 10))

# Model accuracy comparison
plt.subplot(2, 3, 1)
models = ['Decision Tree', 'Random Forest', 'AdaBoost']
accuracies = [dt_accuracy, rf_accuracy, ada_accuracy]
bars = plt.bar(models, accuracies, color=['skyblue', 'lightgreen', 'lightcoral'])
plt.title('Model Accuracy Comparison')
plt.ylabel('Accuracy')
plt.ylim(0, 1)
for bar, acc in zip(bars, accuracies):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
            f'{acc:.3f}', ha='center', va='bottom')

# Feature importance - Random Forest
plt.subplot(2, 3, 2)
top_features_rf = importance_comparison.head(8)
plt.barh(range(len(top_features_rf)), top_features_rf['random_forest'])
plt.yticks(range(len(top_features_rf)), top_features_rf['feature'])
plt.title('Random Forest Feature Importance')
plt.xlabel('Importance')

# Feature importance - AdaBoost
plt.subplot(2, 3, 3)
top_features_ada = importance_comparison.sort_values('adaboost', ascending=False).head(8)
plt.barh(range(len(top_features_ada)), top_features_ada['adaboost'])
plt.yticks(range(len(top_features_ada)), top_features_ada['feature'])
plt.title('AdaBoost Feature Importance')
plt.xlabel('Importance')

# Confusion matrix for Random Forest
plt.subplot(2, 3, 4)
cm_rf = confusion_matrix(y_test, rf_pred)
sns.heatmap(cm_rf, annot=True, fmt='d', cmap='Blues')
plt.title('Random Forest Confusion Matrix')
plt.xlabel('Predicted Unit')
plt.ylabel('Actual Unit')

# Confusion matrix for AdaBoost
plt.subplot(2, 3, 5)
cm_ada = confusion_matrix(y_test, ada_pred)
sns.heatmap(cm_ada, annot=True, fmt='d', cmap='Reds')
plt.title('AdaBoost Confusion Matrix')
plt.xlabel('Predicted Unit')
plt.ylabel('Actual Unit')

# Model agreement analysis
plt.subplot(2, 3, 6)
agreement = (rf_pred == ada_pred).astype(int)
correct_both = ((rf_pred == y_test) & (ada_pred == y_test)).astype(int)
agreement_counts = pd.Series(agreement).value_counts()
plt.pie([agreement_counts[0], agreement_counts[1]], 
        labels=['Disagree', 'Agree'], autopct='%1.1f%%')
plt.title('Model Agreement')

plt.tight_layout()
plt.show()

print("\nRESULT: Ensemble learning models implemented and compared successfully!")

---
# Summary and Conclusion

## Experiments Completed:
1. **Dataset Loading and Analysis** - Successfully loaded and analyzed TNPSC question dataset
2. **Statistical Analysis** - Computed comprehensive statistics and distributions
3. **Linear Regression** - Predicted difficulty scores using question features
4. **Classification Models** - Implemented Bayesian Logistic Regression and SVM
5. **Clustering Analysis** - Applied K-Means and Gaussian Mixture Models
6. **Dimensionality Reduction** - Used PCA for feature analysis and visualization
7. **Decision Trees** - Implemented CART algorithm for question categorization
8. **Ensemble Learning** - Compared Random Forest and AdaBoost performance

## Key Findings:
- The dataset contains questions across 8 units with varying difficulty levels
- Text-based features combined with metadata provide good classification performance
- Ensemble methods generally outperform single classifiers
- PCA reveals meaningful patterns in question characteristics
- Different units have distinct linguistic and structural patterns

## Applications for TNPSC Exam Preparation:
- **Automated Question Classification** - Categorize questions by unit and difficulty
- **Personalized Study Plans** - Recommend questions based on student performance
- **Difficulty Prediction** - Estimate question difficulty for adaptive testing
- **Content Analysis** - Identify key topics and patterns in exam questions
- **Performance Analytics** - Track student progress across different units

---
**Project**: AI-Powered Personalized Exam Preparation Assistant for Competitive Exams  
**All 9 ML Laboratory Experiments Completed Successfully!**